In [1]:
import numpy as np

In [2]:
%pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2

In [4]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

In [6]:
%pip install albumentations

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import albumentations as A

In [8]:
%pip install lungs-segmentation==1.1.1 --no-deps

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from lungs_segmentation.pre_trained_models import create_model

In [10]:
model = create_model("resnet34")

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cpu


In [12]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import lungs_segmentation.inference as inference

In [14]:
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Chest_Xray_PA_3-8-2010.png/739px-Chest_Xray_PA_3-8-2010.png

zsh:1: command not found: wget


In [15]:
import matplotlib.pyplot as plt
import matplotlib

In [16]:
import os

# # 强制将模型加载到 CPU
# model = model.cpu()
# # 调用推理函数
# image, mask = inference.inference(model, '/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/CXR/0.png', thresh=0.2)

# Resize the result image to 1024x1024 before saving

# result_img = inference.img_with_masks(image, [mask[0], mask[1]])

# # 显示结果
# plt.figure(figsize=(20, 40))
# plt.imshow(result_img, cmap="gray")
# Ensure the saved image has the same resolution as the original image
# plt.show()
# Define the directory containing the images
image_dir = '/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/CXR/'

# Iterate over all files in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Create a new directory to save the results
        result_dir = '/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/Mask-ResNet34/'
        os.makedirs(result_dir, exist_ok=True)
        
        # Construct the full file path
        file_path = os.path.join(image_dir, filename)
        
        # Perform inference
        image, mask = inference.inference(model, file_path, thresh=0.2)
        
        # Generate result image with segmentation mask
        result_img = inference.img_with_masks(image, [mask[0], mask[1]])
        result_img = cv2.resize(result_img, (1024, 1024))
        
        # Save the result image in the new directory with the same name as the original image
        result_filepath = os.path.join(result_dir, filename)
        plt.imsave(result_filepath, result_img, cmap="gray")

In [1]:
import os
import cv2
import numpy as np
import albumentations as A

# 设置图像大小和预处理
img_size = 1024
aug = A.Compose([A.Resize(img_size, img_size, interpolation=1, p=1)], p=1)

# 读取图片和mask
def preprocess_image(img_path, mask_path):
    # 读取去骨的 CXR 图像
    cxr_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if cxr_image is None:
        raise FileNotFoundError(f"Image not found at {img_path}")
    
    cxr_image = cv2.normalize(cxr_image, None, 0, 255, cv2.NORM_MINMAX)  # 归一化到0-255范围
    cxr_image = np.array(cxr_image, dtype=np.uint8)  # 确保是 NumPy 数组
    cxr_image_aug = aug(image=cxr_image)["image"]

    # 读取 mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(f"Mask not found at {mask_path}")
    
    mask = np.array(mask, dtype=np.uint8)  # 确保是 NumPy 数组
    mask_aug = aug(image=mask)["image"]

    # 转换 mask 为二值
    binary_mask = (mask_aug > 128).astype(np.uint8)  # 二值化 (0 或 1)
    
    return cxr_image_aug, binary_mask

# 应用 mask 到去骨的胸片上
def apply_mask_to_cxr(cxr_image, binary_mask):
    # 按位与操作，仅保留 mask 中为 1 的区域
    masked_image = cv2.bitwise_and(cxr_image, cxr_image, mask=binary_mask)
    return masked_image

# 文件夹处理函数
def process_folder(bs_folder, mask_folder, output_folder):
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    
    # 遍历去骨胸片图像文件夹
    for filename in os.listdir(bs_folder):
        # 过滤非图像文件
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Skipping non-image file: {filename}")
            continue

        # 构造去骨图像和 mask 的完整路径
        cxr_path = os.path.join(bs_folder, filename)
        mask_path = os.path.join(mask_folder, filename)
        
        # 检查去骨图像和 mask 文件是否存在
        if not os.path.exists(cxr_path):
            print(f"BS image not found for {filename}, skipping...")
            continue
        if not os.path.exists(mask_path):
            print(f"Mask not found for {filename}, skipping...")
            continue
        
        try:
            # 预处理图像和 mask
            cxr_image, binary_mask = preprocess_image(cxr_path, mask_path)

            # 应用 mask
            masked_cxr = apply_mask_to_cxr(cxr_image, binary_mask)

            # 保存结果到输出文件夹
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, masked_cxr)
            print(f"Processed and saved: {output_path}")
        
        except FileNotFoundError as e:
            print(f"File error for {filename}: {e}")
        except Exception as e:
            print(f"Unexpected error for {filename}: {e}")

# 主函数
if __name__ == "__main__":
    # 输入文件夹路径（替换为你的路径）
    bs_folder = "/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/BS"  # 去骨胸片图像文件夹路径
    mask_folder = "/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/Mask-DenseNet121"       # mask 文件夹路径
    output_folder = "/Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121"   # 输出文件夹路径

    # 对整个文件夹进行处理
    process_folder(bs_folder, mask_folder, output_folder)

Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/348.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/412.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/374.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/360.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/406.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/638.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/176.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/88.png
Processed and saved: /Users/bennychan/Desktop/SZCH-X-Rays - Clinical Evaluation/MaskBS-DenseNet121/610.png
Processed and saved: /Users/bennychan/